# Match Crawler

This notebook is responsible for data collection process. The notebook is a crawler for high elo League of Legends matches. Crawling is achieved through the Riot Development API and the package `riotwatcher`.

In [8]:
from riotwatcher import LolWatcher, ApiError
import pandas as pd

import os
import json

api_key = 'RGAPI-2bf73814-83af-4540-a2e0-46eee1c8271f'
watcher = LolWatcher(api_key=api_key)
region = 'kr'

### Fetch Challenger League

Fetch the summoner's in challenger league of Korea server and cache it locally to `data/kr_challenger_league.json`. The last time it was cached is in 2022-05-23.

In [10]:
if os.path.exists('data/kr_challenger_league.json'):
    with open('data/kr_challenger_league.json', 'r') as f:
        kr_challenger_league = json.load(f)
    print("Loaded previously cached data.")
else :
    kr_challenger_league = watcher.league.challenger_by_queue(region, "RANKED_SOLO_5x5")
    with open('data/kr_challenger_league.json', 'w') as f:
        json.dump(kr_challenger_league, f)

Loaded previously cached data.


In [ ]:
counter = 0
challengers = [entry['summonerId'] for entry in kr_challenger_league['entries']]
with open()
for i, summoner in enumerate(challengers):
    if i < counter:
        continue
    